<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Laboratorio práctico: Análisis visual interactivo con Folium**

Tiempo estimado necesario: **40** minutos


La tasa de éxito de un lanzamiento puede depender de muchos factores, como la masa de la carga útil, el tipo de órbita, etc. También puede depender de la ubicación y la proximidad del sitio de lanzamiento, es decir, la posición inicial de las trayectorias del cohete. Encontrar la ubicación óptima para construir un sitio de lanzamiento sin duda implica muchos factores, y esperamos poder descubrir algunos de ellos analizando las ubicaciones de los sitios de lanzamiento existentes.

En los laboratorios de análisis exploratorio de datos anteriores, visualizaste el conjunto de datos de lanzamiento de SpaceX con `matplotlib` y `seaborn` y descubriste algunas correlaciones preliminares entre el sitio de lanzamiento y las tasas de éxito. En este laboratorio, realizarás análisis visuales más interactivos con `Folium`.

## Objectivos


Este laboratorio contiene las siguientes tareas:

* **TAREA 1:** Marcar todos los sitios de lanzamiento en un mapa
* **TAREA 2:** Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa
* **TAREA 3:** Calcular las distancias entre un sitio de lanzamiento y sus proximidades

Después de completar las tareas anteriores, debería poder identificar patrones geográficos en los sitios de lanzamiento.

Primero, importemos los paquetes de Python necesarios para este laboratorio:

In [79]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [80]:
import folium
import pandas as pd

In [81]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

Si necesita refrescar su memoria sobre el folium, puede descargar y consultar este laboratorio de folium anterior:

[Generacion de mapas con python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [82]:
## Tarea 1: Marcar todos los sitios de lanzamiento en un mapa


Primero, intentemos agregar la ubicación de cada sitio en un mapa usando las coordenadas de latitud y longitud del sitio.

El siguiente conjunto de datos con el nombre `spacex_launch_geo.csv` es un conjunto de datos aumentado con latitud y longitud agregadas para cada sitio.

In [83]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Ahora, puedes echar un vistazo a cuáles son las coordenadas de cada sitio.

In [84]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Las coordenadas anteriores son simplemente números que no te dan una idea intuitiva de dónde están esos sitios de lanzamiento. Si eres bueno en geografía, puedes interpretar esos números directamente en tu mente. Si no, no hay problema. Visualicemos esas ubicaciones marcándolas en un mapa.

Primero debemos crear un objeto `Mapa` en forma de folio, con una ubicación central inicial que será el Centro Espacial Johnson de la NASA en Houston, Texas.

In [85]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Podríamos usar `folium.Circle` para agregar un área circular resaltada con una etiqueta de texto en una coordenada específica. Por ejemplo,

In [86]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

y deberías encontrar un pequeño círculo amarillo cerca de la ciudad de Houston y puedes hacer zoom para ver un círculo más grande.

Ahora, agreguemos un círculo para cada sitio de lanzamiento en el marco de datos `launch_sites`

*TODO:* Crear y agregar `folium.Circle` y `folium.Marker` para cada sitio de lanzamiento en el mapa del sitio

Un ejemplo de folium.Circulo:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


Un ejemplo de folio.Marcador:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [87]:
# Inicializar el mapa
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Para cada sitio de lanzamiento, agregue un objeto Círculo basado en sus coordenadas (Latitud, Longitud). 
# Además, agregue el nombre del sitio de lanzamiento como etiqueta emergente.
for fila, columna in launch_sites_df.iterrows():
    sitecoords=columna['Lat'],columna['Long']
    sitename=columna['Launch Site']
    circle = folium.Circle(location=sitecoords,radius=1000,color='blue',fill=True).add_child(folium.Popup(sitename))
    marker = folium.Marker(location=sitecoords,icon=DivIcon(icon_size=(20,20),icon_anchor=(0.0), html='<div style="font-size: 12;color:#d35400;"><b>%s</b></div>' % sitename, ))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map
    

El mapa generado con los sitios de lanzamiento marcados debería verse similar al siguiente:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Ahora, puedes explorar el mapa ampliando o reduciendo las áreas marcadas e intentar responder las siguientes preguntas:

* ¿Están todos los sitios de lanzamiento cerca de la línea del Ecuador?
* ¿Están todos los sitios de lanzamiento muy cerca de la costa?

Intenta también explicar tus hallazgos.

In [88]:
# Tarea 2: Marcar los lanzamientos exitosos/fallidos de cada sitio en el mapa


A continuación, intentemos mejorar el mapa añadiendo los resultados de lanzamiento de cada sitio y veamos cuáles tienen una alta tasa de éxito.
Recuerde que el marco de datos spacex_df contiene registros detallados de lanzamiento y la columna `class` indica si este lanzamiento fue exitoso o no.

In [89]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


A continuación, crearemos marcadores para todos los registros de lanzamiento. Si un lanzamiento fue exitoso `(class=1)`, usaremos un marcador verde; si fue fallido, usaremos un marcador rojo `(class=0)`.

Tenga en cuenta que un lanzamiento solo ocurre en uno de los cuatro sitios de lanzamiento, lo que significa que muchos registros de lanzamiento tendrán exactamente la misma coordenada. Los grupos de marcadores pueden ser una buena manera de simplificar un mapa que contiene muchos marcadores con la misma coordenada.

Primero, creemos un objeto `MarkerCluster`

In [90]:
marker_cluster = MarkerCluster().add_to(site_map)



*TODO:* Crear una nueva columna en el marco de datos `spacex_df` llamada `marker_color` para almacenar los colores del marcador según el valor de `class`

In [91]:
# Asegúrate que marker_cluster ya está añadido a site_map:
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coords = [record['Lat'], record['Long']]
    sitio = record['Launch Site']
    resultado = 'Éxito' if record['class'] == 1 else 'Fallo'
    color_icon = 'green' if record['class'] == 1 else 'red'

    # Crear marcador con popup y color de icono según resultado
    marker = folium.Marker(
        location=coords,
        popup=f"Sitio: {sitio}<br>Resultado: {resultado}",
        icon=folium.Icon(color='white', icon_color=color_icon)
    )
    # Agregar marcafor index, record in spacex_df.iterrows():
    coords = [record['Lat'], record['Long']]
    sitio = record['Launch Site']
    resultado = 'Éxito' if record['class'] == 1 else 'Fallo'
    color_icon = 'green' if record['class'] == 1 else 'red'

    # Crear marcador con popup y color de icono según resultado
    marker = folium.Marker(
        location=coords,
        popup=f"Sitio: {sitio}<br>Resultado: {resultado}",
        icon=folium.Icon(color='white', icon_color=color_icon)
    )
    # Agregar marcador al cluster
    marker_cluster.add_child(marker)

# Mostrar mapa
site_map


*TODO:* Para cada resultado de lanzamiento en el marco de datos `spacex_df`, agregue un `folium.Marker` a `marker_cluster`

Su mapa actualizado puede parecerse a las siguientes capturas de pantalla:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


A partir de los marcadores etiquetados por color en los grupos de marcadores, debería poder identificar fácilmente qué sitios de lanzamiento tienen tasas de éxito relativamente altas.

In [92]:
# TAREA 3: Calcular las distancias entre un sitio de lanzamiento y sus proximidades

A continuación, necesitamos explorar y analizar las proximidades de los sitios de lanzamiento.

Primero, agreguemos una `PosiciónMouse` al mapa para obtener las coordenadas al pasar el ratón sobre un punto. Así, mientras explora el mapa, podrá encontrar fácilmente las coordenadas de cualquier punto de interés (como una vía férrea).

In [93]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Ahora acérquese a un sitio de lanzamiento y explore su proximidad para ver si puede encontrar fácilmente algún ferrocarril, autopista, costa, etc. Mueva el mouse a estos puntos y marque sus coordenadas (que se muestran en la parte superior izquierda) para indicar la distancia al sitio de lanzamiento.

In [94]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Marque un punto en la costa más cercana usando MousePosition y calcule la distancia entre el punto de la costa y el sitio de lanzamiento.

In [95]:
# Encuentra las coordenadas de la línea de costa más cercana
# p. ej.: Lat: 28.56367 Lon: -80.57163
# distancia_costa = calcular_distancia(lat_sitio_lanzamiento, longitud_sitio_lanzamiento, lat_línea_costa, longitud_línea_costa)
# Coordenadas del sitio de lanzamiento
lat1 = 28.56326
lon1 = -80.57673
# Coordenadas del punto en la costa (las que viste con el mouse)
lat2 = 28.56299
lon2 = -80.56795
# Calcular la distancia en kilómetros
distancia_costa = calculate_distance(lat1,lon1,lat2,lon2)

print(f"Distancia desde el sitio de lanzamiento hasta la costa: {distancia_costa:.2f} km")


Distancia desde el sitio de lanzamiento hasta la costa: 0.86 km


In [96]:
# Crea y añade un folium.Marker en el punto de costa más cercano seleccionado en el mapa.
# Muestra la distancia entre el punto de costa y el punto de lanzamiento usando la propiedad icon.
# Por ejemplo:
# distance_marker = folium.Marker(
# coordinate,
# icon=DivIcon(
# icon_size=(20,20),
# icon_anchor=(0,0),
# html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
# )
# )

# Crear un marcador con DivIcon que muestre la distancia en el mapa
distance_marker = folium.Marker(
    location=(lat2, lon2),  # Ubicación del punto en la costa
    icon=DivIcon(
        icon_size=(100, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distancia_costa),
    )
)

# Añadir el marcador al mapa
site_map.add_child(distance_marker)

# Mostrar el mapa actualizado
site_map


*TODO:* Dibujar una `Polilínea` entre un sitio de lanzamiento y el punto de la costa seleccionado

In [97]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
# Coordenadas del sitio de lanzamiento
lat1 = 28.56326
lon1 = -80.57673
# Coordenadas del punto en la costa (las que viste con el mouse)
lat2 = 28.56299
lon2 = -80.56795
# Crear una lista de las coordenadas [inicio, fin]
coordenadas_linea = [(lat1, lon1), (lat2, lon2)]

# Crear la línea
lines = folium.PolyLine(locations=coordenadas_linea, color='blue', weight=2)

# Añadir la línea al mapa (ahora sí)
site_map.add_child(lines)

# Mostrar el mapa
site_map


Su mapa actualizado con la línea de distancia debería verse como la siguiente captura de pantalla:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* De manera similar, puedes dibujar una línea entre un sitio de lanzamiento y su ciudad, ferrocarril, autopista, etc. más cercano. Primero debes usar la `Posición del mouse` para encontrar sus coordenadas en el mapa.

Un símbolo de un mapa ferroviario podría verse así:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


Un símbolo en un mapa de carreteras podría verse así:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


Un símbolo en un mapa de una ciudad podría verse así:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [101]:
# Crea un marcador con la distancia a la ciudad, vía férrea, autopista, etc. más cercana.
# Dibuja una línea entre el marcador y el punto de lanzamiento.
# Supongamos que ya tienes estas variables definidas:
# lat1, lon1: coordenadas del sitio de lanzamiento
# lat2, lon2: coordenadas del punto cercano (ciudad, autopista, etc.)

# Coordenadas del sitio de lanzamiento
lat1 = 28.56326
lon1 = -80.57673

# Coordenadas del punto cercano (ciudad, autopista, etc.)
lat3 = 28.07883
lon3 = -80.60481

# Calculamos la distancia usando tu función calculate_distance (devuelve km)
distancia = calculate_distance(lat1, lon1, lat3, lon3)

# Crear marcador en el sitio de lanzamiento
launch_marker = folium.Marker(
    location=(lat1, lon1),
    popup='Sitio de lanzamiento',
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
)

# Crear marcador en el punto cercano con la distancia como etiqueta
nearby_marker = folium.Marker(
    location=(lat3, lon3),
    popup=f'Distancia: {distancia:.2f} km',
    icon=folium.DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color: green;"><b>{distancia:.2f} KM</b></div>'
    )
)

# Crear línea entre los dos puntos
line = folium.PolyLine(
    locations=[(lat1, lon1), (lat3, lon3)],
    color='green',
    weight=3,
    opacity=0.7
)

# Añadir los elementos al mapa
site_map.add_child(launch_marker)
site_map.add_child(nearby_marker)
site_map.add_child(line)

# Mostrar el mapa
site_map


Después de trazar las líneas de distancia a las proximidades, puede responder fácilmente a las siguientes preguntas:

* ¿Están los sitios de lanzamiento cerca de vías férreas?
* ¿Están los sitios de lanzamiento cerca de carreteras?
* ¿Están los sitios de lanzamiento cerca de la costa?
* ¿Los sitios de lanzamiento se mantienen a cierta distancia de las ciudades?

También, intente explicar sus hallazgos.

¿Están los sitios de lanzamiento cerca de vías férreas?
Sí.
La mayoría de los sitios de lanzamiento están relativamente cerca y ayuda a porder transportar cohetes y suministros pesados desde fábricas.

¿Están los sitios de lanzamiento cerca de carreteras?
Sí.
En los mapas se puede observar que hay **carreteras (por ejemplo, Centaur Road o State Highway 3) muy cercanas a los sitios de lanzamiento. Esto permite el acceso terrestre para personal, vehículos, suministros.

¿Están los sitios de lanzamiento cerca de la costa?
Sí.
La mayoría de los sitios están ubicados a escasa distancia del mar, en torno a 0.5 a 1 km.
Permite que los lanzamientos se realicen sobre el océano, reduciendo riesgos sobre zonas pobladas.
Si algo falla durante el lanzamiento, los escombros caen al mar.

¿Los sitios de lanzamiento se mantienen a cierta distancia de las ciudades?
Sí.
Los sitios de lanzamiento están relativamente alejados de ciudades como Cocoa o Titusville (varios kilómetros).
Minimiza el riesgo en caso de explosión.
Reduce el impacto acústico en zonas residenciales.



# Próximos pasos:

Ahora has descubierto mucha información interesante sobre la ubicación de los sitios de lanzamiento usando folium, de forma muy interactiva. A continuación, tendrás que crear un panel con Ploty Dash basado en registros detallados de lanzamiento.

## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
